In [1]:
import pandas as pd
import pandasql as ps

In [3]:
import string
import gc
import time

In [4]:
cms = pd.read_csv('cms.csv')
cms = cms[['Ref Series Title', 'Genre']]
shows = list(set(cms[(cms['Genre'] == 'Kids') | (cms['Genre'] == 'Kids,Animation')].reset_index()['Ref Series Title']))

In [5]:
query = pd.read_csv('file.csv', encoding="ISO-8859-1")[['search_string', 'count(search_string)', 'sum(number_of_search_results)']]

c:\users\aakasht\appdata\local\continuum\anaconda3\envs\py361\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
### Get the queries for which zero results were showed
query = query[query['sum(number_of_search_results)'] == 0].reset_index()[['search_string', 'count(search_string)']]

In [7]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [8]:
def filter_noneng(s):
    if isEnglish(s):
        return s

In [9]:
query_ = []
query_instances = []
for i,q in enumerate(query['search_string']):
    if type(filter_noneng(q)) == str:
        query_.append(filter_noneng(q))
        query_instances.append(query['count(search_string)'][i])

In [10]:
from multiprocessing import Pool

In [20]:
query_1 = query_[:int(len(query_)/4)]
query_2 = query_[int(len(query_)/4):int(len(query_)/4)*2]
query_3 = query_[int(len(query_)/4)*2:int(len(query_)/4)*3]
query_4 = query_[int(len(query_)/4)*3:]

In [21]:
query_instances_1 = query_instances[:int(len(query_instances)/4)]
query_instances_2 = query_instances[int(len(query_instances)/4):int(len(query_instances)/4)*2]
query_instances_3 = query_instances[int(len(query_instances)/4)*2:int(len(query_instances)/4)*3]
query_instances_4 = query_instances[int(len(query_instances)/4)*3:]

In [14]:
from final_func import *

In [22]:
if __name__ == '__main__':
    pool = Pool(processes=4)
    result_list = pool.map(final_results, [(query_1, shows, query_instances_1), (query_2, shows, query_instances_2), (query_3, shows, query_instances_3), (query_4, shows, query_instances_4)])

In [30]:
final_tags = None
for i in range(4):
    if i == 0:
        final_tags = pd.DataFrame(result_list[i])
    else:
        final_tags = final_tags.append(pd.DataFrame(result_list[i]))

In [32]:
final_tags.columns = ['query', 'show_name', 'volume_of_faulty_query']

In [38]:
final_tags = final_tags[['show_name', 'query', 'volume_of_faulty_query']].sort_values(['volume_of_faulty_query'], ascending = False).reset_index(drop =True)

In [39]:
final_tags.to_csv('final_kids_tags_feb_2_apr_2018.csv', header = True)

In [ ]:
cost_fn('MRKP', 'POKEMON')